# Direct and inverse geometry of 2d robots



## Displaying objects

Let's first learn how to open a 3D viewer, in which we will build our simulator. First start gepetto-gui. Best is to run it directly from the shell by typing gepetto-gui. A new window with the Gepetto logo should open. Objects can be now created from the python commands.

The following GView object is a client of the Gepetto Viewer server, i.e. it will be use to pass display command to the viewer. The first commands are to create objects, 

In [85]:
import gviewserver
gv = gviewserver.GepettoViewerServer()

gv.addSphere ('world/ball',    .1,         [1 ,0 ,0,1])  # radius, color=[r,g,b,1]
gv.addCapsule('world/capsule', .05,.75,    [1 ,1 ,1,1])  # radius, length, color = [r,g,b,1]
gv.addBox    ('world/box',     .2,.05,.5,  [.5,.5,1,1]) # depth(x),length(y),height(z), color

True

Execute the above python commands once, you get a "True" output. Execute it a second time, you get a False: that's just telling you that the object world/box already exists and Gepetto viewer cannot create it again. If you want to erase your world and all your objects, just run:


In [86]:
gv.deleteNode('world', True)  # name, all=True

You now have to run again the gviewerserver.GepettoViewerServer() command to create the world again.


In [87]:
gv = gviewserver.GepettoViewerServer()

Placing objects can be done using the applyConfiguration command, and specifying the placement as a 3D translation and quaternion rotation. Don't forget to refresh your window after placing your objects.

In [88]:
gv.applyConfiguration('world/box',  [.1,.1,.1,  1,0,0,0 ]) # x,y,z, quaternion
gv.refresh()

In a first time, we will work in 2D. Here is a shortcut to place an object from x,y,theta 2d placement.

In [89]:
import numpy as np
def placement(x,y,theta): return [ y, 0, x,  0,np.sin(theta/2), 0, np.cos(theta/2) ]
gv.applyConfiguration('world/capsule', placement(0.1,.2, np.pi/4))
gv.refresh()

## Creating a 2d robot
This robot will have 2 joints, named shoulder and elbow, with link of length 1 to connect them. First let's create the 5 geometry objects.

In [90]:
gv.addSphere ('world/joint1',    .1,    [1 ,0 ,0,1])
gv.addSphere ('world/joint2',    .1,    [1 ,0 ,0,1])
gv.addSphere ('world/joint3',    .1,    [1 ,0 ,0,1])
gv.addCapsule('world/arm1', .05,.75,    [1 ,1 ,1,1])
gv.addCapsule('world/arm2', .05,.75,    [1 ,1 ,1,1])

True

Given a configuration vector q of dimension 2, compute the position of the centers of each object, and display correctly the robot.

In [91]:
from pinocchio.utils import *
q = rand(2)

In [92]:
# %load solution_display_2r.py
c0 = np.cos(q[0,0]); s0 = np.sin(q[0,0])
c1 = np.cos(q[0,0]+q[1,0]); s1 = np.sin(q[0,0]+q[1,0])

gv.applyConfiguration('world/joint1',  placement(0,0,0) )
gv.applyConfiguration('world/arm1',    placement(c0/2,s0/2,q[0,0]) )

gv.applyConfiguration('world/joint2',  placement(c0,s0,q[0,0]) )
gv.applyConfiguration('world/arm2',    placement(c0+c1/2,s0+s1/2,q[0,0]+q[1,0]) )

gv.applyConfiguration('world/joint3',  placement(c0+c1,s0+s1,q[0,0]+q[1,0]) )

gv.refresh()


## Optimize the configuration 
Scipy is a collection of scientific tools for Python. It contains, in particular, a set of optimizers that we are going to use for solving the inverse-geometry problem. If not done yet, install scipy with sudo "apt-get install python-scipy"

In [93]:
# %load example_scipy.py


For now, let's use the simpler BFGS (unconstrained) solver. Define a cost function denoting the distance from the robot end-effector to an arbitrary target. 

In [94]:
# %load solution_optimize_q.py



## What configuration to optimize?
It seems logical to optimize over the angles $q_1,q_2$. However, other representations of the configuration are possible. Consider for example the explicit representation, where the placement of each body 1,2,3 is stored. For each body, we get $x,y,\theta$, so 9 parameters in total. In addition, each body position is constrained with respect to the placement of the previous body, with 6 constraints in total. 

What are the pros and cons? The effector position is now a trivial function of the representation, hence the cost function is very simple. The trade-off is that we have to explicitly satisfy the constraints. 

Let's start by defining the configuration.

In [95]:
x1,y1,th1,  x2,y2,th2,  x3,y3,th3 = x = np.zeros(9)

The cost function is now just a sparse difference on x3,y3:

In [96]:
target = [ .5, .5 ]
def cost_9(x):
    x1,y1,th1,  x2,y2,th2,  x3,y3,th3 = x 
    return (x3-target[0])**2+(y3-target[1])**2

The constraint function should return a vector, each coefficient corresponding to one of the 6 constraints:

In [97]:
def constraint_9(x):
    x1,y1,th1,  x2,y2,th2,  x3,y3,th3 = x 
    from numpy import cos, sin
    return np.array([
        x1 - 0,
        y1 - 0,
        x1 + cos(th1) - x2,
        y1 + sin(th1) - y2,
        x2 + cos(th2) - x3,
        y2 + sin(th2) - y3,
    ])

In [98]:
x0 = np.zeros(9)
print(cost_9(x0), constraint_9(x0))

(0.5, array([0., 0., 1., 0., 1., 0.]))


The callback function is now accepting dimension 9 vector:

In [100]:
def callback_9(ps):
    '''Display the robot in Gepetto Viewer. '''
    assert(ps.shape == (9,) )
    x1,y1,t1, x2,y2,t2, x3,y3,t3 = ps
    gv.applyConfiguration('world/joint1',  placement(x1,y1,t1) )
    gv.applyConfiguration('world/arm1',    placement(x1+np.cos(t1)/2, x1+np.sin(t1)/2, t1))
    gv.applyConfiguration('world/joint2',  placement(x2,y2,t2) )
    gv.applyConfiguration('world/arm2',    placement(x2+np.cos(t2)/2, y2+np.sin(t2)/2, t2))
    gv.applyConfiguration('world/joint3',  placement(x3,y3,t3) )
    gv.refresh()
    import time; time.sleep(.5)
callback_9(x0)

The BFGS solver defined above cannot be used directly to optimize over equality constraints. A dirty trick is to add the constraint as a penalty, i.e. a high-weigth term in the cost function: $penalty(x) = cost(x) + 100*||constraint(x)||^2$ . Here, we are in a good case where the optimum corresponds to the 0 of both the constraint and the cost. The penalty with any weight would lead to the optimum and perfect constraint satisfaction. Yet the solver suffers to reach the optimum, because of the way we have described the constraint.

In [110]:
# %load solution_optimize_placements_bfgs.py


Alternatively, the solver S-LS-QP (sequential least-square quadratic-program) optimizes over equality constraints.

In [109]:
# %load solution_optimize_placements.py

When properly defining the constraint, the solver converges quickly. It is difficult to say a-priori whether it is better to optimize with the q (and consequently a dense cost and no constraint) or with the x-y-theta (and consequently a sparse cost and constraints). Here, we empirically observe no difference. 